<a href="https://colab.research.google.com/github/mmorari-cmyk/CURSO_SE.ALES_Y_SISTEMAS/blob/main/Taller2/DESHBOARD_1_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.8 MB/s eta 0:00:00


In [3]:
%%writefile THD.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
st.sidebar.title("Presentado por:")
st.sidebar.write("Marlyn Nathalia Mora Riascos")


# Título y descripción general
st.title("Aplicación de Circuitos Electrónicos: Cálculo del THD en Circuitos RC y Resistivos Puros")

st.write("""
Esta aplicación presenta el **análisis y cálculo de la Distorsión Armónica Total (THD)** en circuitos
resistivos puros y en circuitos RC, utilizando herramientas de procesamiento digital de señales.
""")

# Sección teórica
st.write("""
### ¿Qué es el THD?

La **Distorsión Armónica Total (THD)** cuantifica el nivel de distorsión presente en una señal eléctrica,
comparando la energía de los armónicos con la de su componente fundamental.
Se expresa como la relación entre la raíz cuadrada de la suma de los cuadrados de los armónicos
y la magnitud del armónico fundamental.
""")

st.latex(r"THD = \frac{\sqrt{V_2^2 + V_3^2 + V_4^2 + \cdots}}{V_1}")

st.write("""
Donde:

- $V_1$ es la magnitud de la **frecuencia fundamental**.
- $V_2, V_3, V_4, \\ldots$ representan las magnitudes de los **armónicos superiores** de la señal.
""")

# Explicación del método
st.write("""
### Cálculo del THD mediante la Transformada Rápida de Fourier (FFT)

Para determinar el THD, se aplica la **Transformada Rápida de Fourier (FFT)**, la cual convierte la señal
del dominio del tiempo al dominio de la frecuencia.
De este modo, es posible identificar el componente fundamental (de mayor magnitud) y los armónicos que lo acompañan.

Una vez obtenidas las amplitudes de los armónicos, se calcula el THD usando la fórmula anterior.
Este procedimiento permite evaluar la calidad de una señal y el efecto de los componentes no lineales en el circuito.
""")


# Definición de los parámetros de la señal de entrada
Fo = 60   # Frecuencia fundamental de la señal (Hz)
Fs = 30 * Fo  # Frecuencia de muestreo, 30 veces mayor que la fundamental (Hz)
To = 1 / Fo   # Duración de un periodo fundamental (s)
Ts = 1 / Fs   # Intervalo entre muestras o periodo de muestreo (s)
t = np.arange(0, 5 * To, Ts)  # Vector de tiempo que cubre 5 periodos de la señal
A = 120  # Valor de amplitud máxima de la señal (V)


# Generación de la señal de entrada y proceso de rectificación
st.subheader("Generación y Rectificación de la Señal de Entrada")
st.write("""
Se crea una señal senoidal de **60 Hz**, característica de los sistemas eléctricos de corriente alterna.
Posteriormente, dicha señal se somete a un proceso de **rectificación**,
el cual permite representar el comportamiento de la tensión tras pasar por un circuito rectificador.
""")
in_o = A * np.sin(2 * np.pi * Fo * t)
rec_c = sig.square(2 * np.pi * Fo * t)  # Tren de pulsos para la señal rectificada completa
in_ = in_o * rec_c  # Entrada rectificada completa

# Mostrar las gráficas de la señal de entrada y rectificada
st.write("### Señal de Alimentación Original (Senoidal)")
plt.figure()
plt.plot(t, in_o, label='Alimentación', color="orange")
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt)

st.write("### Señal Rectificada Completa")
plt.figure()
plt.plot(t, in_, label='Señal Rectificada', color="red")
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt)

# Controles interactivos para los parámetros del circuito RC
st.subheader("Ajuste de Parámetros del Circuito RC")
st.write("""
Utiliza los controles deslizantes para modificar los valores de la **resistencia (R)** y la **capacitancia (C)**,
y analiza cómo estos parámetros influyen en el comportamiento de la señal rectificada a la salida del circuito RC.
""")
R_v = st.slider('Resistencia R (Ohmios)', min_value=100, max_value=2000, value=1000, step=100)
C_v = st.slider('Capacitancia C (μF)', min_value=1, max_value=50, value=10, step=1) * 1e-6

# Función de transferencia del circuito RC en serie
st.write("""
### Función de Transferencia del Circuito RC en Serie
Para el circuito RC en serie, la función de transferencia se define como:
""")
st.latex("H(s) = \\frac{1}{RCs + 1}")


st.write("""
Esta función de transferencia permite modelar el comportamiento del circuito en el dominio de la frecuencia.
""")
# Definición de la función de transferencia del circuito RC
num = np.array([1])  # Coeficiente del numerador
den = np.array([R_v * C_v, 1])  # Coeficientes del denominador
G_n = sig.TransferFunction(num, den)  # Definición de la función de transferencia

# Evaluar la salida del circuito RC
_, out_rc, _ = sig.lsim(G_n, U=in_, T=t)

# Gráfica de la salida del circuito RC
st.subheader("Salida del Circuito RC")
st.write("""
La siguiente gráfica muestra cómo la señal rectificada cambia al pasar por el circuito RC en serie.
""")
plt.figure()
plt.plot(t, out_rc, label='Salida del Circuito RC')
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt)
#plt.plot(t, in_,label='In(t)')#entrada rectificada
st.subheader("Salida del Circuito resistivo")
st.write("""
La siguiente gráfica muestra cómo la señal rectificada de salida del circuito resistivo.
""")
plt.figure()
plt.plot(t, in_,c="r",label='Out(t)')#salida del circuito
plt.xlabel('t [s]')
plt.ylabel('out(t)')
st.pyplot(plt)
# Cálculo del THD simplificado basado en la FFT
def calcular_thd(signal, Fs):
    Xf = np.fft.rfft(signal)  # FFT de la señal
    magnitudes = np.abs(Xf)
    freqs = np.fft.rfftfreq(len(signal), 1 / Fs)

    # Identificar la frecuencia fundamental
    fundamental_idx = np.argmax(magnitudes)
    fundamental = magnitudes[fundamental_idx]

    # Cálculo del THD como la suma de los armónicos sobre la fundamental
    thd = np.sqrt(np.sum(magnitudes[1:]**2)) / fundamental
    return thd

# Calcular el THD
thd_rc = calcular_thd(out_rc, Fs)
thd_resistive = calcular_thd(in_, Fs)

# Mostrar el valor del THD
# Resultados del cálculo de THD y factor de potencia
st.subheader("Resultados del THD")
st.markdown("""
El **Total Harmonic Distortion (THD)** o **Distorsión Armónica Total** cuantifica el nivel de distorsión
presente en una señal debido a la aparición de armónicos.
A continuación, se presentan los valores calculados del THD para ambos casos de estudio:
""")

st.markdown(f"**THD del circuito RC**: {thd_rc * 100:.2f}%")
st.markdown(f"**THD del circuito resistivo puro**: {thd_resistive * 100:.2f}%")

# Cálculo del Factor de Potencia a partir del THD
st.subheader("Cálculo del Factor de Potencia basado en el THD")

st.subheader("¿Qué es el Factor de Potencia?")
st.write("""
El **Factor de Potencia (FP)** expresa la eficiencia con la que un sistema eléctrico convierte la energía
en trabajo útil.
Cuando la señal presenta **distorsión armónica (THD)**, parte de la potencia se distribuye en los armónicos,
reduciendo la eficiencia del sistema.

La relación entre el factor de potencia y el THD se define mediante la siguiente expresión:
""")

st.latex(r'''
FP = \sqrt{\frac{1}{1 + \left(THD\right)^2}}
''')

st.write("""
En esta fórmula, el **THD** representa la proporción de distorsión de la señal.
A medida que el valor del THD aumenta, el factor de potencia disminuye, indicando una menor eficiencia
en la transferencia de energía eléctrica.
""")

st.markdown("""
A continuación, se muestran los valores calculados del **factor de potencia** para el circuito resistivo puro
y para el circuito RC en serie:
""")

pf1 = np.sqrt(1 / (1 + (thd_rc )**2))
pf2 = np.sqrt(1 / (1 + (thd_resistive)**2))
st.markdown(f"**FP del circuito RC**: {pf1:.4f}")
st.markdown(f"**FP del circuito resistivo puro**: {pf2:.4f}")

# FFT de la señal de salida para el circuito RC
vfre_rc = np.fft.rfftfreq(len(out_rc), Ts)  # Vector de frecuencias en Hz
Xf_rc = np.fft.rfft(out_rc)  # FFT de la salida

st.markdown("### FFT de la Salida del Circuito RC")
st.markdown("""
La **Transformada de Fourier** nos muestra el contenido de frecuencias de la señal.
La siguiente gráfica muestra cómo se distribuyen las frecuencias en la señal después de pasar por el circuito RC.
""")
plt.figure()
plt.stem(vfre_rc, np.abs(Xf_rc) / len(out_rc))
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Magnitud FFT')
st.pyplot(plt)

# FFT de la señal para el circuito resistivo puro
vfre_resistive = np.fft.rfftfreq(len(in_), Ts)
Xf_resistive = np.fft.rfft(in_)

st.write("### FFT del Circuito Resistivo Puro")
st.write("""
Este gráfico muestra la FFT de la señal en un circuito resistivo puro, sin ningún circuito adicional que altere la señal.
""")
plt.figure()
plt.stem(vfre_resistive, np.abs(Xf_resistive) / len(in_))
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Magnitud FFT')
st.pyplot(plt)


Overwriting THD.py


In [4]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run THD.py &>/content/logs.txt & #Cambiar  ...py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-11-09 13:54:47--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-11-09 13:54:47--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-09T14%3A37%3A55Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-09